In [5]:
from prody import *
from pylab import *
import numpy as np
import py3Dmol

#import nglview as nv 
# ^^ For use in Jupyter Notbook
#from IPython.display import display, Math, Latex
# ^^ For use in Jupyter Notbook
#import sys

# Program: AnisotropicNetworkModel.ipynb

# Author(s): Originally Jose K. James; edited, rewritten, and adopted for further use by Jan A. Siess.

# Purpose: The purpose of this program is to conduct normal analysis of proteins via the use of --
# an anisotropic network model. ANM is a tool used for normal analysis of proteins, and has been --
# successfully applied for exploring the relation between between protein structure and function. 
# It is essentially an Elastic Network Model for the Cα atoms with a step function -- 
# for the dependence of the force constants on the inter-particle distance.

# Python Version: Intended for Python versions 3.[*]+


prot = 'drosophila_mutopt_r3_backbone.nowa.avg'

#if in regular terminal working with a regular [*].py file:
#prot = sys.argv[1]

# Parse structure of PDB by passing an identifier. If it is not located in the current working --
# directory, it will be downloaded.
avg = parsePDB('avg.pdb')

#Extracting only C$\alpha$ atoms.
calphas = avg.select('protein and name CA')

#Get the actual number of alpha carbon atoms into a variable
num_CA = calphas.numAtoms('protein')

print(f'The number of Carbon Atoms is: {num_CA}')
#display(Math(r'F(k) = \int_{-\infty}^{\infty} f(x) e^{2\pi i k} dx'))

@> 568 atoms and 1 coordinate set(s) were parsed in 0.01s.


The number of Carbon Atoms is: 568


In [2]:
#Building the Hessian

#Instantiate an ANM instance:
anm = ANM('avg ANM analysis')

#Build the Hessian matrix by passing the selected @>568 atoms to ANM.buildHessian() method:
anm.buildHessian(calphas, cutoff = 13.5, gamma = 1.0)

#Get a copy of the Hessian matrix using ANM.getHessian() method [further, we limit full output to only round up to 3 places]:
structHessian = anm.getHessian().round(3)

#Just out of curiousity to determine the number of element in the array:
SquareMatrix = np.prod(structHessian.shape)
print(f'The number of array elements in the Hessian matrix amounts to: {SquareMatrix}')

#Setting the force constant K to 1.0 kcal/Å^{2} and the cutt-off distance to 13.5 Å 
#This was determined from previously derived general ranges for proteins.
CO = 13.5
K = 1*10**0

#Save the array to a text file 
np.savetxt(f'{prot}_Hessian_gamma_{K}_cutoff_{CO}.dat', structHessian)

@> Hessian was built in 0.15s.


The number of array elements in the Hessian matrix amounts to: 2903616


In [3]:
#Obtaining the normal modes data 

#The normal mode of an oscillating system is a pattern of motion in which all parts of the system move sinusoidally with the same frequency and with a fixed phase relation.

#n_modes -- number of non-zero eigenvalues/vectors to calculate
#zeros -- if True, modes with zero eigencalues will be kept
#turbo -- Use a memory intensive, but faster way to calculate modes

#determine first the number of n_modes
num_modes = num_CA*3 

#Go ahead with the main calculation...

#Calculating Normal Modes, eigenvalues/-vectors, and the covariance matrix:
anm.calcModes(n_modes=num_modes, zeros = False, turbo = True)
eigVals = anm.getEigvals()
eigVects = anm.getEigvecs()
egCov = anm.getCovariance()

#Save the eigenvalue/-vectors into a txt file:
np.savetxt(f'{prot}_egvals2_gamma_{K}_cutoff_{CO}.dat', eigVals)
np.savetxt(f'{prot}_egvects2_gamma_{K}_cutoff_{CO}.dat', eigVects)
np.savetxt(f'{prot}_egCov2_gamma_{K}_cutoff_{CO}.dat', egCov)




@> 1698 modes were calculated in 0.68s.
